Part 1: Libraries installation

In [1]:
#Install following libraries
!pip install biopython
!pip install primer3-py

SyntaxError: invalid syntax (1579938633.py, line 2)

Part 2: Import modules

In [2]:
#import following modules from installed libaries.
import Bio
from Bio.Seq import Seq
import primer3
from Bio.SeqUtils.MeltingTemp import Tm_NN
#tkinter is required to save file at defined location. it is default library in python.
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import sys
import os

Part 3: Primer design

In [3]:
#Set a definition for the fast cloning methods: FC1, method to create chimera using two PCRs, 1st for vector, and second for insert. The scheame for FC1 is Vect1-INSERT-Vect2.FC2, method is to insert/delete an small sequence (<100) from a target region. FC2 will generate one pair of primers.
#
def program_FC1():
    #Inputs....
    #Ask for insert sequence.
    construct_input = input("""Paste here your chimera construct <<vector Part1 + Insert + 
                            Vector part2>>. 
                            \n
                            Please use only + symbol to separate the sequences. \n
                            The vector part should be more than 40bps and insert 100bp or more""", )
    
    # defined split symbol
    marker = '+'

    #example sequence
    #construct_input = """ATGAAAGAGAAATTAAACAGACTGCTATTCACGTCGAAAACGAGAATGCGTGTTTTTAGCAAATTATCCAGGTATCTAAGTAATGGTGTTCCAGTAACGTTTGCCCTAGCTGAGCTTTATAAGTTTACGTCTGATGAGGGAAGAAAAAAAGATAATCCGGATGCATTTGCCCTGCAAAGATGGTTAATTGCTGTAAGAAACGGGAAAACATTGGCTGAAGCGATGAGGGGGTGGGTACCTTTTGACGAACTGTCAATAATTTCCGCCGGGGAAATCTCAGGTAATGTTCACCAGGCGTTGGATGATATCATTTATATGAATGATACAAAAAAGAAAGTAAAA   +  ggcgcactggcagggattatttatcctgtagtcctgcttctgacgacatgtctgtatttgcatatatttggaactcaggttgttccggcattttcaggcatcctgcctgtagagaaatggcagggcgcaggcaggactatgtattatcttgctgtattcgttcaggattatcttgtcattacactgctgtcttttatgatggtgatattattaatactggca  +  ACTCTGTCAAGATGGACCGGAAGGTTGCGTCTCTTCTTTGACAGATTTATTCCCTGGTCGATATACAAAACCATTATAGGATGTGGTTTCTTGTTATCACTGGCATCGCTTATTAATGCAGGTATCCCTGTACCGGAAGCCTTACGAATAATAATGAAAACGGCAAGTCCGTGGTATAAGGAAAGATTAGTTGCTATAAGATACAGATTATTAAATGGAGAGAGAAATCTTGGGGAGGCCCTTAATTCATCTGGTTACATTTTCCCATCCAAACAGATGATAATGGATCTCCGTTCTTATGCTGCACTTGGAGGCTTTGATGAAATGCTTAATAAACTTTCAGTTCAATGGCAGGATGACAGTGTTGCTTATATAACGAAACAAATGACT"""

    ## Removed any kind of gap from the input sequence.
    cons_str = construct_input.replace("\n", "").replace(' ','').replace('-','')
    cons_seq = Seq(cons_str) # to convert into sequence
      

    # Split the input string from the split symbol "+".
    sequences = cons_seq.split(marker)

    # Confirm if there are exactly three sequences, if not print error.
    if len(sequences) != 3:
        print("Error: Please provide exactly three sequences separated by the specified marker.")
    else:
        # Extract and define the individual sequences
        vectP1, insert, vectP2 = sequences

    # Calculate the "insert" sequence length.
    len_insert = int(len(insert))
    if len_insert <=80:
        warning1 = "Insert length is less than 80bp, you may want use Primer based insertion and select method 2"
    else:
        warning1 = None

    #Create the final construct by adding sequence as vect_p1 + Insert + vect_p2, also add NNN to avoid any issue during translation into protein sequence in the next step.
    
    
    cons_seq_gap = vectP1+insert.lower()+vectP2 #The constructed sequence, may have any gap!# lower casing insert for user.
    
    #Remove if there is any gap present.
    cons_seq = Seq(cons_seq_gap).replace(" ", "").replace("\n", "")

    cons_seq_trans = cons_seq + Seq("NNN") #Added "NNN" at the end, it required only for translation.

#To convert the sequence into Fasta formate with row length 120 bp.
    def format_fasta(sequence, line_length=60):
        fasta_string = ">Sequence\n"
        for i in range(0, len(sequence), line_length):
            line = sequence[i:i + line_length]
            fasta_string += line + "\n"
        return fasta_string

   
    # Format as FASTA with a line length of 120
    fasta_data = format_fasta(cons_seq, line_length=120)

    
    # Default Tm range
    default_min_tm = 55
    default_max_tm = 65

    # Ask the user if they want to customize Tm values
    customize_tm = input(f"[Default Tm: {default_min_tm}-{default_max_tm}], Do you want to customize the Tm range? (yes/no): ")

    if customize_tm.lower() == "yes":
        # If the answer is yes, ask for desired min and max Tm values
        tm_range = input("Please provide your 'min-max' Tm range, eg. 60-65")
        min_tm, max_tm = tm_range.split('-')
        desired_min_tm = float(min_tm)
        desired_max_tm = float(max_tm)
    else:
        # Use the default values
        desired_min_tm = default_min_tm
        desired_max_tm = default_max_tm


    #Generate translation of the final construct to see, if it is what you expected. This is not niccessory for the fastcloning primer designing but could be useful for user if they want to do protein work.
        
    #Translate frame 1 for the final construct.
    frame1 = cons_seq_trans[0:].translate()

    #translate frame 2 for the final construct
    frame2 = cons_seq_trans[1:].translate()

    #translate frame 3 for the final construct
    frame3 = cons_seq_trans[2:].translate()

    # Format as FASTA with a line length of 120
    fasta_frame1 = format_fasta(frame1, line_length=120)
    fasta_frame2 = format_fasta(frame2, line_length=120)
    fasta_frame3 = format_fasta(frame3, line_length=120)        

    #### Mathmetical parts to do calculation for primer designing.
    #First need to find the location (loc1) to start the primers
    loc1 = int(len(vectP1)) #Location to start the Reverse Primer for vector amplification.
    loc1n = loc1-26 # 26 is a number for the minimum length of primers matching region.
    loc2 = loc1+ int(len(insert))

    #Lets design the reverse Primer for vector amplifications.
    #Vector Revers Primer = pVR.

    x = 1 # Want to increase the length by x to achieve required Tm.

    pVectR = cons_seq[(loc1n-x):loc1] # Initial reverse primer sequence, this will change later. 

    # Calculate the initial Tm value
    current_tm = round(Tm_NN(cons_seq[(loc1n-x):loc1]))

    # Check if the current Tm is outside the desired range
    while current_tm < desired_min_tm or current_tm > desired_max_tm:
        if current_tm < desired_min_tm:
            x += 1  # Increment x to increase Tm
        else:
            x -= 1  # Decrement x to decrease Tm

        # Calculate the new Tm value
        current_tm = round(Tm_NN(cons_seq[(loc1n-x):loc1]))

    # Now x contains the value that meets the desired Tm range
    pVRs = cons_seq[(loc1n-x):loc1]
    pVR = pVRs.reverse_complement() #This is the final Reverse primer for Vector.
    tmR = round(Tm_NN(pVR)) # Tm for the primer
    pVR_len = len(pVR) # Length for the primer.

    ############################################
    #Forword Primer for vector amplifications
    #Vector  Forward Primer = pFR.

    y = 1 # Want to increase the length by y to achieve required Tm.
    loc3 = loc2+26 #26 is a number for the minimum length of primers matching region.
    pVFi = cons_seq[loc2:(loc3+y)] # Initial Forward primer for Vector.
    #print(pVFi)
    # Calculate the initial Tm value for F primer
    current_tmF = round(Tm_NN(cons_seq[loc2:(loc3+y)]))

    # Check if the current Tm is outside the desired range
    while current_tm < desired_min_tm or current_tmF > desired_max_tm:
        if current_tmF < desired_min_tm:
            y += 1  # Increment y to increase Tm
        else:
            y -= 1  # Decrement y to decrease Tm

        # Calculate the new Tm value
        current_tmF = round(Tm_NN(cons_seq[loc2:(loc3+y)]))

    # Now y contains the value that meets the desired Tm range
    pVF = cons_seq[loc2:(loc3+y)] # The final Forward primer for Vector. 
    #print(pVF)
    tmf = Tm_NN(cons_seq[loc2:(loc3+y)]) # forward primer Tm
    pVF_len = len(pVF) # Forward primer length.
    
    
    ###################################################
    ###Design primers for the insert sequence

    #Farword Primer for Insert = pIF.

    z = 1 #length for matching region sequence from the insert sequence.
    ########################################
    pIFs = cons_seq[(loc1-16):(loc1+18+z)] # Assumed pIF. It has 16bp from the vectP1 region (from the 3' ends to 16 bp upstream) and from the 3' ends to 22bp + z bp upstream in the insert seq. (loc1+ 22+ z).

    # Calculate the initial Tm value for th primerIF
    current_tmIF = round(Tm_NN(cons_seq[loc1:(loc1+22+z)]))
    # Check if the current Tm is outside the desired range
    while current_tmIF < desired_min_tm or current_tmIF > desired_max_tm:
        if current_tmIF < desired_min_tm:
            z += 1  # Increment z to increase Tm
        else:
            z -= 1  # Decrement z to decrease Tm
        # Ensure z is never less than 0 (minimum matching length of 18 bp)
        if z < 0:
            z = 0
        # Calculate the new Tm value
        current_tmIF = round(Tm_NN(cons_seq[loc1:(loc1+22+z)]))


    # Now z contains the value that meets the desired Tm range

    pIF = cons_seq[(loc1-16):(loc1+22+z)]
    tmIf = current_tmIF#round(Tm_NN(cons_seq[loc1:(loc1+z)]),2) #Tm specific for matching region to insert.
    tmIf2 = round(Tm_NN(pIF),2) #tm for total length of primer


    ##Reverse Primer for Insert = pIR.
    w = 1 #length for matching region sequence from the insert sequence.
    pIRs = cons_seq[(loc2-22+w):(loc2+16)] # Assumed pIR, It has 22+ w bp from the insert 3' sequenece region + 16bp from VectP2 5' region. The w will get adujusted to meet the Tm.

    # Calculate the initial Tm value for the pIR
    current_tmIR = round(Tm_NN(cons_seq[(loc2-22+w):loc2]))
    # Check if the current Tm is outside the desired range
    while current_tm < desired_min_tm or current_tmIR > desired_max_tm:
        if current_tmIR < desired_min_tm:
            w += 1  # Increment w to increase Tm
        else:
            w -= 1  # Decrement w to decrease Tm
        # Ensure w is never less than 0 (minimum matching length of 18 bp)
        if w < 0:
            w = 0
        # Calculate the new Tm value
        current_tmIR = round(Tm_NN(cons_seq[(loc2-22+w):loc2]))

    pIRt = cons_seq[(loc2-22+w):(loc2+16)]
    pIR = pIRt.reverse_complement() #The final sequence for pIR.
    tmIr = current_tmIR # Tm value for the matching sequences from Insert region.
    tmIr2 = round(Tm_NN(pIR),2) #tm for total length of primer

    # For input data Format as FASTA with a line length of 120
    fasta_vectP1 = format_fasta(vectP1, line_length=120)
    fasta_insert = format_fasta(insert, line_length=120)
    fasta_vectP2 = format_fasta(vectP2, line_length=120)

    # Your results
    results_FC1 = f"""Thanks for using FastCloneAssist Primer Tool!\n
    Report:\nPlease find pairs of Primers to do your Fast_cloning PCR.\n
    Warring: {warning1}
    \n

    Vector specific Primer Pair:-
    ======================================================

        Vect.Primer_F:
        5' {pVF} 3'
        Primer length: {len(pVF)}
        Tm is: {round(tmf, 2)} Degree Celsius

        Vect.Primer_R:
        5' {pVR} 3'
        Primer length: {len(pVR)}
        Tm is: {round(tmR, 2)} Degree Celsius

    Insert specific Primer Pair:-
    ======================================================
        
        InsertPrimer_F:
        5' {pIF} 3'
        Primer length: {len(pIF)}
        Tm for matching to insert seq is: {round(tmIf, 2)} Degree Celsius
        Tm for full-length is: {tmIf2} Degree Celsius

        InsertPrimer_R:
        5' {pIR} 3'
        Primer length: {len(pIR)}
        Tm for matching to insert seq is: {round(tmIr, 2)} Degree Celsius
        Tm for full-length is: {tmIr2} Degree Celsius
    ======================================================

    The new construct sequence: The inserted sequece are in lower case for easy viewing.
        \n{fasta_data}

    Protein translations of the Final constructs: 

        Frame 1: \n {fasta_frame1}\n
        Frame 2: \n {fasta_frame2}\n
        Frame 3: \n {fasta_frame3} \n
        -------- For Record Keeping ----------\n
        
    Your Input sequences are copied below in the format of- 
        \n'Vector_part1+INSERT+Vector_part2':\n

        Vector Part 1:
        {fasta_vectP1}\n

        Insert Sequence:
        {fasta_insert}\n

        Vector Part 2:
        {fasta_vectP2}
        """

    #Print results as display and save in downloads
    print(results_FC1)

    #To save as Text file in download
    from google.colab import files

# Generate your report content as a string
    report_content1 = results_FC1

# Create a text file object
    report_file = open("FastCloning1_primers.txt", "w")
    report_file.write(report_content1)
    report_file.close()

# Download the file
    files.download("FastCloning1_primers.txt")   
#########################################
#########################################
##########################################
#Define the FC2 program. This is for Primer based deletion or insertion or both (max insertion sequence 120bp).
def program_FC2():
    # Inputs, Sequences in organised format and Tm (optional)
    # Ask for input, here user will define the deletion region by * symbols as well as if there is insert add * and your insert. see below example. And if there is no deletion only insertion, mark the insertion point by two stars, **, and provide the insert sequence as above.
    #construct_inputs = """AGATCTCGATCCCGCGAAATTAATACGACTCACTATAGGGGAATTGTGAGCGGATAACAATTCCCCTCTAGAAATAAGGGCGGTGCGTGTGCCGCCTTTGGGG**CGAGCGGGCCGTGTCGGGGGGAATTAATACGACTCACTATAGGGGAATTGTGAGCGGATAACAATTCCCCTCTAGAAATAATTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTATCATCACAGCAGCGGCCTGGTGCCGCGCGGCAGCCATATG *AAAAAAAAAAAAAAAAAAAAAAAAAA"""
    construct_inputs = input("""Input sequence, Decorate your sequence with three * symbols. 
                             \n
                             First two stars to mark deletion sequence boundaries or the insertion location when no deletion needed. The third star at the end of the target sequence. If there is an insert sequence, it will go after the 3rd star.
                             \n 
                             Example 1: ...ATGCATGC*TTTTT*ATGCATGC...*AAAAA, here TTTTT is the deletion target and AAAAA is the insertion seq. 
                             \n
                             Example 2: ...ATGC**TGAC...*AAAAA, ** is the insertion location for the AAAAA seq. 
                             \n
                             Example 3: ...ATGC*AAAA*GTCA...*, here the AAAA is selected for deletion without any insertion""",
                             )
    
    
    ## Remove any gap or line gap from the sequence.
    cons_str = construct_inputs.replace("\n", "").replace(" ","").replace("-","")
    #convert string input into a sequence file
    cons_seq = Seq(cons_str)
    
    # Split the input string into parts using the * symbol
    parts = cons_seq.split('*')
    

    # Initialize variables to store the sequences
    partP1 = ""
    deleted_seq = ""
    partP2 = ""
    insertP = ""

    # Check the number of parts, if missing generate a worning to the user.
    if len(parts) == 3:
        partP1, deleted_seq, partP2 = parts
    elif len(parts) == 4:
        partP1, deleted_seq, partP2, insertP = parts
    else:
        print("Error: Please provide either 3 or 4 sequences separated by the specified markers.")

    if len(insertP) >= 120:
        warning = "Warning!! The insert is bigger than 120bp"
    else:
        warning = None  # No warning if the condition is not met

#To convert the sequence into Fasta formate with row length 120 bp.
    def format_fasta(sequence, line_length=60):
        fasta_string = ">Sequence\n"
        for i in range(0, len(sequence), line_length):
            line = sequence[i:i + line_length]
            fasta_string += line + "\n"
        return fasta_string

    #change part1 and part2 into fasta formate
    fasta_partP1 = format_fasta(partP1, line_length = 120)
    fasta_partP2 = format_fasta(partP2, line_length = 120)
    
    ##########################################################
    #primer designing: Mathmetical portion

    #create the chimera by deleting the region specified (the * marking) and adding the insert sequence there.
    geneM_gap = partP1 + insertP.lower() + partP2 #used lower font style to separate the insert, will be helpful to user.
    
    #Remove any gap further
    geneM = Seq(geneM_gap).replace("\n", "").replace(" ", "").replace("-","")

    geneM_trans = geneM + Seq("NNN") #Adding "NNN" will make translation by the python error free. 

# Format as FASTA with a line length of 120
    fasta_geneM = format_fasta(geneM, line_length=120)

    #Translate the modified gene into protein to show, if the insertion is as you wanted.
    #translate frame 1 for the final construct.
    frame1 = geneM_trans[0:].translate()
    #translate frame 2 for the final construct
    frame2 = geneM_trans[1:].translate()
    #translate frame 3 for the final construct
    frame3 = geneM_trans[2:].translate()
    #print(geneM_trans, '\n', frame1, '\n',frame2,'\n', frame3)
    
    #convert translaation into fasta formate
    fast_frame1 = format_fasta(frame1, line_length = 120)
    fast_frame2 = format_fasta(frame2, line_length = 120)
    fast_frame3 = format_fasta(frame3, line_length = 120)



    #Input sequences length calculations

    partP1_len = int(len(partP1)) # length of part 1, the value is required to do primer start-end calculations.
    
    p1l = m1 = partP1_len #making short variable.
    #find the middle of inserted sequence, it is required to make the primers start & end points.
    #calculate the length fo insert
    insertP_len = len(insertP) #insert length (il) (insert nucleotide sequence).
    
    il = insertP_len #made the variable short.
    if il % 2 == 0:  #if the division by 2 is not an integer make an integer by adding 1.
        il_m = int(il / 2) #to calculate the middle of insert.
    else:
        il_m = int((il + 1) / 2) #Half of insert seq length: l2
    midP = m2 = int(p1l + il_m) # middle position of the insert in the modified sequence, geneM.

    oE = m3= int(m2 + (il-il_m)) # oE, other End is the start point of gene after the insert sequence in the geneM.

    #print(p1l, il,il_m, midP,oE)

    # Default Tm range
    default_min_tm = 55
    default_max_tm = 65

    # Ask the user if they want to customize Tm values
    customize_tm = input(f"Do you want to customize the Tm range? (yes/no) [Default: {default_min_tm}-{default_max_tm}]: ")

    if customize_tm.lower() == "yes":
        # If the user wants to customize, ask for desired min and max Tm values
        tm_range = input("Please provide your 'min-max' Tm range, eg. 60-65")
        min_tm, max_tm = tm_range.split('-')
        desired_min_tm = float(min_tm)
        desired_max_tm = float(max_tm)
    else:
        # Use the default values
        desired_min_tm = default_min_tm
        desired_max_tm = default_max_tm

    ########################
    ############################
    #Forward primer (p1f) design....

    x = 1  # starting with 18bp, but it will change as per Tm needs.
    #Forward primer_match region seq, see below remarks.
    
    p1f = geneM[m3:(m3 +26+(x))]

    # Calculate the initial Tm value for F primer
    current_tm_p1f = round(Tm_NN(geneM[m3:(m3 +26 + (x))]))
    # Check if the current Tm is outside the desired range
    while not (desired_min_tm <= current_tm_p1f <= desired_max_tm):
        if current_tm_p1f < desired_min_tm:
            x += 1  # Increment x to increase Tm
        else:
            x -= 1  # Decrement x to decrease Tm

        # Ensure x is never less than 0 (minimum matching length of 18 bp)
        if x < -8:
            x = 7

        # Calculate the new Tm value
        current_tm_p1f = round(Tm_NN(geneM[m3:(m3 +26+(x))]))
    p1fs = geneM[m3:(m3 +26+(x))]
    #print(p1f)
    #print(len(p1f))
    #print(p1fs)
    #print(len(p1fs))
    #print(current_tm_p1f)
    #print(x)

    #(pmf), Forward primer's region between overhang(p2f) and match(p1f),
    pmf = geneM[m2:m3]  #this length will vary according to the insert length.
    #Forward primer_overhang (p2f) = geneM[(m2-8):m2]
    p2f = geneM[(m2 - 8):m2]

    #Now get the complete forward primer by adding the p2f and p1f part.
    primerF = p2f + pmf + p1fs

    #primer length
    pf_l = len(primerF)
    primerFtm = round(Tm_NN(primerF), 2) #for full length
    #print(primerF)
    #print(pf_l,primerFtm)
    #print(current_tm_p1f)

    #Revers primer designing, same as above, only need to convert as complementry seq at the end.

    
    y = 1  # staring with 1, but it will change as per Tm needs.

    #Reverse primer_match (p1r)
    p1r = geneM[(m1 -(26 + y)):m1]

    # Calculate the initial Tm value for F primer
    current_tm_p1r = round(Tm_NN(geneM[(m1 -(26 + y)):m1]))
    # Check if the current Tm is outside the desired range
    while not (desired_min_tm <= current_tm_p1r <= desired_max_tm):
        if current_tm_p1r < desired_min_tm:
            y += 1  # Increment x to increase Tm
        else:
            y -= 1  # Decrement x to decrease Tm

        # Ensure x is never less than 0 (minimum matching length of 18 bp)
        if y < -8:
            y = 7

        # Calculate the new Tm value
        current_tm_p1r = round(Tm_NN(geneM[(m1 -(26 + y)):m1]))
    p1rs = geneM[(m1 -(26 + y)):m1]
    #print(p1r)
    #print(len(p1r))
    #print(p1rs)
    #print(len(p1rs))
    #print(current_tm_p1r)
    #print(y)

    #(pmr), Reverse primer's region between overhang(p2r) and match(p1rs),
    pmr = geneM[m1:m2]  #this length will vary according to the insert length.

    #Revers primer_overhang (p2r) = geneM[m2:(m2+8)]
    p2r = geneM[m2:(m2 + 8)] # 8pb should be matched
    #print(p2r)
    primerR = (p1rs + pmr + p2r).reverse_complement()
    #print(f"Reverse primer: 5' {primerR} 3'")
    #primer length
    pr_l = len(primerR)
    #primerRtm = round(Tm_NN(primerR[(pr_l - y):]), 2)
    primerR_tm = round(Tm_NN(primerR),2)
    #print(primerR_tm)
    #print("Reverse primer length is: ", (pr_l), "bp")
    #Generate reports
    # Your results
    results_FC2 = (
        """Thanks for using FastCloneAssist Primer Tool!\n\n"""
        f"Report: Your primers for performing Fast Cloning PCR to either delete, insert, or delete and insert at a specified location..\n\n"
        f"Warning: {warning}\n\n"
        f"=====================================================================\n\n"
        f"Forward primer:\n 5' {primerF} 3'\n"
        f"Tm based on total length of the primer is: {primerFtm} Degree Celsius\n"
        f"Tm for matching seq is: {current_tm_p1f} Degree Celsius\n"
        f"Forward primer length is: {pf_l} bp\n\n"
        f"Reverse primer:\n 5' {primerR} 3'\n"
        f"Tm based on total length of the primer is: {primerR_tm} Degree Celsius\n"
        f"Tm for matching seq is: {current_tm_p1r} Degree Celsius\n"
        f"Reverse primer length is: {pr_l} bp\n"
        f"=====================================================================\n\n"
        f"The modified Chimera gene sequence:\n{fasta_geneM}\nlength = {(len(geneM))}bp\n\n"
        f"Final Sequence Translation, Frame 1:\n{fast_frame1}\n\n"
        f"Final Sequence Translation, Frame 2:\n{fast_frame2}\n\n"
        f"Final Sequence Translation, Frame 3:\n{fast_frame3}\n\n"
        f"----------For The Record Keeping-------------\n"
        f"Your Input sequence after partition are: \nSequence Part 1: \n{fasta_partP1}\n\n"
        f"Sequence Part 2: \n{fasta_partP2} \n\n"
        f"The chosen deletion region length = {(len(deleted_seq))}bp \nDeleted sequence: \n{deleted_seq}\n\n"
        f"The provided insert sequence is:\n{insertP}\nlength = {(len(insertP))}bp\n\n"
        
        
    )
#Print results as display and save in downloads
    print(results_FC2)

    #To save as Text file in download
    from google.colab import files

# Generate your report content as a string
    report_content2 = results_FC2

# Create a text file object
    report_file = open("FastCloning2_primers.txt", "w")
    report_file.write(report_content2)
    report_file.close()

# Download the file
    files.download("FastCloning2_primers.txt") 

    
#And you can make def for your program now it can be asked as choice,..

choice = input("""\nWELCOME to MD:FastCloning_PrimerTool!\nSelect your Method of Fast cloning.\nEnter 1, if you want to PCR amplify your Vector and Insert separately, you will get 2 pairs of primers.\nEnter 2, if you want to do primer based insertion (<120bp) or/and deletion""",)

if choice == "1":
    program_FC1()
elif choice == "2":
    program_FC2()
else:
    print("Invalid choice. Please select 1 or 2.")

C:\Users\pksingh\AppData\Roaming\Python\Python38\site-packages\Bio\Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Results saved to C:/Users/pksingh/OneDrive/01_A-My projects/12_Primer_designing Paper/Data/test_FC_Tools/aprli_24_Class1_2.txt
